In [41]:
import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools, offline
plotly.tools.set_credentials_file(username='mtodisco10', api_key='bCfUmq5FralbymKXY4uX')
import re
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

In [103]:
%run progress_bar.ipynb
%run field_layout.ipynb

In [43]:
#Read in Data
game_data = pd.read_csv('game_data.csv')
play_info = pd.read_csv('play_information.csv')
play_player_role = pd.read_csv('play_player_role_data.csv')
player_punt = pd.read_csv('player_punt_data.csv')
video_footage_control = pd.read_csv('video_footage-control.csv')
video_footage_injury = pd.read_csv('video_footage-injury.csv')
video_review = pd.read_csv('video_review.csv')

In [44]:
def read_NGS_data(file_lst):
    for i, file in enumerate(file_lst):
        print('Reading in {}'.format(file))
        data = pd.read_csv(file)
        if i == 0:
            NGS_df = data
            del data
        else:
            NGS_df = pd.concat([NGS_df, data])
            del data
    return NGS_df

In [45]:
file_lst = ['NGS-2016-pre.csv','NGS-2016-reg-wk1-6.csv','NGS-2016-reg-wk7-12.csv','NGS-2016-reg-wk13-17.csv',\
            'NGS-2016-post.csv'#,'NGS-2017-pre.csv','NGS-2017-reg-wk1-6.csv','NGS-2017-reg-wk7-12.csv',\
#            'NGS-2017-reg-wk13-17.csv','NGS-2017-post.csv'
           ]

In [46]:
NGS_df = read_NGS_data(file_lst)

Reading in NGS-2016-pre.csv
Reading in NGS-2016-reg-wk1-6.csv
Reading in NGS-2016-reg-wk7-12.csv
Reading in NGS-2016-reg-wk13-17.csv
Reading in NGS-2016-post.csv


### Injury Plays EDA

In [47]:
'There are concussion injuries on ' + str(round(len(video_review) / float(len(play_info)) * 100, 2)) + '% of ' + 'punt plays'

'There are concussion injuries on 0.55% of punt plays'

##### It is good for the NFL and players that injuries only occur on less than 1% of punt plays, but it will be hard to do analysis

#### How are players being injured?

In [48]:
trace1 = go.Bar(
        x=video_review.groupby(['Player_Activity_Derived'], as_index=False)['PlayID'].count()['Player_Activity_Derived'],
        y=video_review.groupby(['Player_Activity_Derived'], as_index=False)['PlayID'].count()['PlayID']
    )
trace2 = go.Bar(
        x=video_review.groupby(['Primary_Impact_Type'], as_index=False)['PlayID'].count()['Primary_Impact_Type'],
        y=video_review.groupby(['Primary_Impact_Type'], as_index=False)['PlayID'].count()['PlayID'],
    )
trace3 = go.Bar(
        x=video_review.groupby(['Friendly_Fire'], as_index=False)['PlayID'].count()['Friendly_Fire'],
        y=video_review.groupby(['Friendly_Fire'], as_index=False)['PlayID'].count()['PlayID'],
    )
trace4 = go.Bar(
        x=video_review.groupby(['Primary_Partner_Activity_Derived'], as_index=False)['PlayID'].count()['Primary_Partner_Activity_Derived'],
        y=video_review.groupby(['Primary_Partner_Activity_Derived'], as_index=False)['PlayID'].count()['PlayID'],
    )

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Player Activity Derived', 'Primary Impact Type',
                                                          'Friendly Fire', 'Primary Partner Activity Derived'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(showlegend=False)

py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



### Quick Conclusions
    - No player activity stands out
    - Helmet to player is primary impact type
    - Friendly fire is only clear 16% of the time
    - No Partner Activity Stands Out

#### Who's getting hurt?

In [49]:
#Categorize player punt roles into the sides of the ball
return_roles = ['PDL1','PDL2','PDL3','PDL4','PDL5','PDL6','PDM','PDR1','PDR2','PDR3','PDR4','PDR5','PDR6'
                ,'PFB','PLL','PLL1','PLL2','PLL3','PLM','PLM1','PLR','PLR1','PLR2','PLR3','PR','VL','VLi'
                ,'VLo','VR','VRi','VRo']

coverage_roles = ['GL','GLi','GLo','GR','GRi','GRo','P','PC','PLG','PLS','PLT','PLW','PPL','PPLi','PPLo'
                 ,'PPR','PPRi','PPRo','PRG','PRT','PRW']

gunner_blockers = ['VL','VLi','VLo','VR','VRi','VRo']

In [50]:
inj_players = video_review.merge(play_player_role, how='inner', on=['Season_Year', 'GameKey', 'PlayID', 'GSISID'])
inj_players.rename(columns={'Role':'inj_role'}, inplace=True)

In [51]:
inj_players['inj_side_of_ball'] = np.where(inj_players.inj_role.isin(return_roles), 'return',
                                          np.where(inj_players.inj_role.isin(coverage_roles), 'coverage', ''))

In [52]:
partner_players = video_review[['Season_Year', 'GameKey', 'PlayID', 'Primary_Partner_GSISID']]
partner_players['Primary_Partner_GSISID'] = partner_players.loc[:,'Primary_Partner_GSISID'] \
                                                .replace('Unclear','0').fillna(0).astype(int)
partner_players = partner_players.merge(play_player_role, how='left', \
                                        left_on=['Season_Year', 'GameKey', 'PlayID', 'Primary_Partner_GSISID'],\
                                       right_on =['Season_Year', 'GameKey', 'PlayID', 'GSISID'])
partner_players = partner_players.drop('GSISID', axis=1)
partner_players.rename(columns={'Role':'partner_role'}, inplace=True)
partner_players['partner_side_of_ball'] = np.where(partner_players.partner_role.isin(return_roles), 'return',
                                          np.where(partner_players.partner_role.isin(coverage_roles), 'coverage', ''))
inj_partner_df = pd.concat([inj_players,partner_players[['partner_role','partner_side_of_ball']]], axis = 1)

In [53]:
inj_partner_df.head()

,Season_Year,GameKey,PlayID,GSISID,Player_Activity_Derived,Turnover_Related,Primary_Impact_Type,Primary_Partner_GSISID,Primary_Partner_Activity_Derived,Friendly_Fire,inj_role,inj_side_of_ball,partner_role,partner_side_of_ball
0,2016,5,3129,31057,Tackling,No,Helmet-to-body,32482,Tackled,No,PLW,coverage,PR,return
1,2016,21,2587,29343,Blocked,No,Helmet-to-helmet,31059,Blocking,No,GL,coverage,PLL1,return
2,2016,29,538,31023,Tackling,No,Helmet-to-body,31941,Tackled,No,GR,coverage,PR,return
3,2016,45,1212,33121,Tackling,No,Helmet-to-body,28249,Tackled,No,PRT,coverage,PR,return
4,2016,54,1045,32444,Blocked,No,Helmet-to-body,31756,Blocked,Yes,PRT,coverage,GR,coverage


In [54]:
partner_players.partner_side_of_ball.value_counts()

return      18
coverage    15
             4
Name: partner_side_of_ball, dtype: int64

In [161]:
ex_play = NGS_df.loc[(NGS_df.Season_Year==2016) & (NGS_df.GameKey== 234) \
                     & (NGS_df.PlayID== 3278) & (NGS_df.GSISID== 28620)].sort_values('Time')

In [174]:
ex_play.Event.value_counts()

punt_downed    1
punt           1
play_submit    1
ball_snap      1
Name: Event, dtype: int64

In [162]:
ball_snap_time = ex_play.loc[ex_play.Event == 'ball_snap'].Time.min()

In [163]:
ex_play = ex_play.loc[ex_play.Time >= ball_snap_time]

In [165]:
ret_play = NGS_df.loc[(NGS_df.Season_Year==2016) & (NGS_df.GameKey== 234) \
                     & (NGS_df.PlayID== 3278) & (NGS_df.GSISID== 27860)].sort_values('Time')
ret_play = ret_play.loc[ret_play.Time >= ball_snap_time]

In [237]:
x = np.array(ex_play.x)
y = np.array(ex_play.y)
xx = np.array(ex_play.x)
yy = np.array(ex_play.y)

In [238]:
x1 = np.array(ret_play.x)
y1 = np.array(ret_play.y)
xx1 = np.array(ret_play.x)
yy1 = np.array(ret_play.y)

In [246]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

init_notebook_mode(connected=True)


data=[dict(x=x, y=y, 
           name='injured player',
           mode='lines', 
           line=dict(width=2, color='blue')
          ),
#       dict(x=x, y=y, 
#            mode='lines', 
#            line=dict(width=2, color='blue')
#           ),
      dict(x=x1, y=y1, 
           name = 'partner player',
           mode='lines', 
           line=dict(width=2, color='blue')
          ),
#       dict(x=x1, y=y1, 
#            mode='lines', 
#            line=dict(width=2, color='blue')
#           )
    ]

layout = load_field()
layout['hovermode'] = 'closest'
layout['updatemenus'] = [{'type': 'buttons',
                           'buttons': [{'label': 'Play',
                                        'method': 'animate',
                                        'args': [None]}]}]

N = len(x)
frames=[dict(data=[dict(x=[xx[k]], 
                        y=[yy[k]], 
                        mode='markers', 
                        marker=dict(color='blue', size=10)
                        )
                   , dict(x=[xx1[k]], 
                        y=[yy1[k]], 
                        mode='markers', 
                        marker=dict(color='orange', size=10)
                        )]
            ) for k in range(0, N, 5)]
          
figure1=dict(data=data, layout=layout, frames=frames)
iplot(figure1)

#### On what types of plays are players getting hurt?

In [70]:
#Create a column in the play_info dataframe to determine the type of play
play_info['outcome'] =  np.where(play_info['PlayDescription'].str.contains('aborted|Fumbled snap|FUMBLES, and recovers', flags=re.IGNORECASE, regex=True), 'aborted',
                        np.where(play_info['PlayDescription'].str.contains('fake|pass|right end|left end|up the middle|Direct snap|right guard', flags=re.IGNORECASE, regex=True), 'fake',
                        np.where(play_info['PlayDescription'].str.contains('muffs', flags=re.IGNORECASE, regex=True), 'muff',         
                        np.where(play_info['PlayDescription'].str.contains('fair catch by', flags=re.IGNORECASE, regex=True), 'fair_catch',
                        np.where(play_info['PlayDescription'].str.contains('touchback', flags=re.IGNORECASE, regex=True), 'touchback',
                        np.where(play_info['PlayDescription'].str.contains('blocked|deflected', flags=re.IGNORECASE, regex=True), 'blocked',
                        np.where(play_info['PlayDescription'].str.contains('out of bounds.', flags=re.IGNORECASE, regex=False), 'oob',
                        np.where(play_info['PlayDescription'].str.contains('downed', flags=re.IGNORECASE, regex=True), 'downed', 
                        np.where(play_info['PlayDescription'].str.contains('safety', flags=re.IGNORECASE, regex=True), 'safety',
                        np.where(play_info['PlayDescription'].str.contains('[0-9]+ for [-+]?[0-9]+ yards?|for no gain|touchdown|(to [A-Z]+ [0-9]+ for [-+]?[0-9]+ yards?)|(to [0-9]+ for [-+]?[0-9]+ yards?)', flags=re.IGNORECASE, regex=True), 'return',         
                        np.where(play_info['PlayDescription'].str.contains('- no play|delay of game|false start, declined|penalty enforced', flags=re.IGNORECASE, regex=True), 'no_play', ' ')))))))))))

In [71]:
pi = play_info.merge(video_review[['Season_Year', 'GameKey', 'PlayID','GSISID']], how='left', on =['Season_Year', 'GameKey', 'PlayID'])
pi['injury'] = np.where(pi.GSISID.notnull(), 1, 0).astype(int)
pi.drop('GSISID', axis = 1, inplace=True)
vr = play_info.merge(video_review, how='inner', on =['Season_Year', 'GameKey', 'PlayID'])

In [72]:
vr.outcome.value_counts()

return        29
downed         3
fair_catch     2
muff           2
fake           1
Name: outcome, dtype: int64

#### Conclusion: players are getting hurt on returns 

In [73]:
#Is it a higher proportion compared to all plays?

In [74]:
pi_inj_grouped = pi.groupby(['outcome'], as_index=False)['injury'] \
    .agg({'total_plays':'count','injuries':sum}) \
    .sort_values('total_plays', ascending = False) \
    .reset_index(drop=True)

pi_inj_grouped['injury_percentage'] = round(pi_inj_grouped['injuries'] / pi_inj_grouped['total_plays'] * 100, 1).astype(str) + '%'
pi_inj_grouped

,outcome,total_plays,injuries,injury_percentage
0,return,2741,29,1.1%
1,fair_catch,1659,2,0.1%
2,downed,796,3,0.4%
3,oob,639,0,0.0%
4,touchback,407,0,0.0%
5,muff,203,2,1.0%
6,no_play,139,0,0.0%
7,fake,45,1,2.2%
8,blocked,39,0,0.0%
9,aborted,9,0,0.0%


#### Conclusion: over 1% of punt plays with a return have an injury
    - **** This is 10x higher than when a fair catch is called

#### Plot Player Paths on a Given Play

In [76]:
# Loading and plotting functions

def load_plays_for_game(GameKey):
    """
    Returns a dataframe of play data for a given game (GameKey)
    """
    play_information = pd.read_csv('play_information.csv')
    play_information = play_information[play_information['GameKey'] == GameKey]
    return play_information


def load_game_and_ngs(ngs_file=None, GameKey=None):
    """
    Returns a dataframe of player movements (NGS data) for a given game
    """
    if ngs_file is None:
        print("Specifiy an NGS file.")
        return None
    if GameKey is None:
        print('Specify a GameKey')
        return None
    # Merge play data with NGS data    
    plays = load_plays_for_game(GameKey)
    ngs = pd.read_csv(ngs_file, low_memory=False)
    merged = pd.merge(ngs, plays, how="inner", on=["GameKey", "PlayID", "Season_Year"])
    return merged


def plot_play(game_df, PlayID, player1=None, player2=None, custom_layout=False):
    """
    Plots player movements on the field for a given game, play, and two players
    """
    game_df = game_df[game_df.PlayID==PlayID]
    
    GameKey=str(pd.unique(game_df.GameKey)[0])
    HomeTeam = pd.unique(game_df.Home_Team_Visit_Team)[0].split("-")[0]
    VisitingTeam = pd.unique(game_df.Home_Team_Visit_Team)[0].split("-")[1]
    YardLine = game_df[(game_df.PlayID==PlayID) & (game_df.GSISID==player1)]['YardLine'].iloc[0]
    
    traces=[]   
    if (player1 is not None) & (player2 is not None):
        game_df = game_df[ (game_df['GSISID']==player1) | (game_df['GSISID']==player2)]
        for player in pd.unique(game_df.GSISID):
            player = int(player)
            trace = go.Scatter(
                x = game_df[game_df.GSISID==player].x,
                y = game_df[game_df.GSISID==player].y,
                name='GSISID '+str(player),
                mode='markers'
            )
            traces.append(trace)
    else:
        print("Specify GSISIDs for player1 and player2")
        return None
    
    if custom_layout is not True:
        layout = load_field()
        layout['title'] =  HomeTeam + \
        ' vs. ' + VisitingTeam + \
        '<br>Possession: ' + \
        YardLine.split(" ")[0] +'@'+YardLine.split(" ")[1]
    data = traces     
    fig = dict(data=data, layout=layout)
    play_description = game_df[(game_df.PlayID==PlayID) & (game_df.GSISID==player1)].iloc[0]["PlayDescription"]
    print("\n\n\t",play_description)
    return py.iplot(fig, filename='jupyter-table1')
    #return offline.iplot(fig)

In [77]:
#game280 = load_game_and_ngs('NGS-2016-reg-wk7-12.csv',GameKey=3278)

In [78]:
#plot_play(game_df=game280, PlayID=2918, player1=32120, player2=32725)

#### Calculate hang time of punts

In [79]:
def get_hang_time(ngs_df, start_event='punt', *stop_events):
    punt_event = ngs_df.loc[ngs_df.Event==start_event] \
        .groupby(['Season_Year', 'GameKey','PlayID'], as_index = False)['Time'].min()
    punt_event.rename(columns = {'Time':'punt_time'}, inplace=True)
    punt_event['punt_time'] = pd.to_datetime(punt_event['punt_time'],\
                                             format='%Y-%m-%d %H:%M:%S.%f')
    
    receiving_event = ngs_df.loc[ngs_df.Event.isin(stop_events)] \
        .groupby(['Season_Year', 'GameKey','PlayID'], as_index = False)['Time'].min()
    receiving_event.rename(columns = {'Time':'receiving_time'}, inplace=True)
    receiving_event['receiving_time'] = pd.to_datetime(receiving_event['receiving_time'],\
                                             format='%Y-%m-%d %H:%M:%S.%f')
    
    punt_df = punt_event.merge(receiving_event, how='inner', on = ['Season_Year','GameKey','PlayID']) \
                .reset_index(drop=True)
    
    punt_df['hang_time'] = (punt_df['receiving_time'] - punt_df['punt_time']).dt.total_seconds()
    
    return punt_df

In [80]:
punt_df = get_hang_time(NGS_df, 'punt', 'punt_received', 'fair_catch')

In [81]:
data = [go.Histogram(x=punt_df.hang_time)]

py.iplot(data, filename='basic histogram')

In [82]:
round(punt_df['hang_time'].mean(), 1)

4.5

In [83]:
round(punt_df['hang_time'].median(), 1)

4.5

In [84]:
str(round(len(punt_df.loc[punt_df.hang_time < 5.5]) / len(punt_df) * 100, 1)) + '% of hang times are less than 5 1/2 seconds'

'96.2% of hang times are less than 5 1/2 seconds'

### Calculate distance between returner and closest coverage man

In [85]:
#Create a function to calculate the space between the returner and the closest coverage player for every second between the punt and the catch
def coverage_returner_space(play_df, ngs_df):
    cov_ret_lst = []
    play_df = play_df.loc[(play_df.outcome == 'fair_catch') | (play_df.outcome == 'return')].reset_index(drop=True)
    for i in log_progress(range(0, len(play_df)), every=25):
        season_key = play_df['Season_Year'][i]
        game_key = play_df['GameKey'][i]
        play_id = play_df['PlayID'][i]
        outcome = play_df['outcome'][i]
        injury = play_df['injury'][i]
        if game_key in ngs_df.GameKey and play_id in ngs_df.PlayID:
            filtered_play = ngs_df.loc[(ngs_df.GameKey == game_key) \
                                       & (ngs_df.PlayID == play_id)].sort_values('Time').reset_index(drop=True)
            filtered_play = filtered_play.merge(play_player_role, \
                                                how='inner', on = ['Season_Year','GameKey','PlayID','GSISID'])
            if len(filtered_play) > 0:
                filtered_play['Time'] = pd.to_datetime(filtered_play['Time'], \
                                                       format='%Y-%m-%d %H:%M:%S.%f')
                punt_event_time = filtered_play.loc[filtered_play.Event == 'punt'].Time.min()
                receiving_event_time = filtered_play.loc[(filtered_play.Event == 'punt_received') | \
                                                         (filtered_play.Event == 'fair_catch')].Time.min()
                gunner_blocker_count = len(filtered_play.loc[filtered_play['Role'].isin(gunner_blockers)]['Role'].unique())
                filtered_play = filtered_play.loc[(filtered_play.Time >= punt_event_time) & \
                                                  (filtered_play.Time <= receiving_event_time)]
                coverage_df = filtered_play.loc[filtered_play['Role'].isin(coverage_roles)].sort_values('Time')
                coverage_df.rename(columns={'x':'cov_x', 
                                          'y': 'cov_y',
                                          'GSISID': 'cov_GSISID',
                                          'dis': 'cov_dis',
                                          'o': 'cov_o',
                                          'dir': 'cov_dir',
                                          'Role': 'cov_Role'
                                         }, inplace=True)
                
                returner_df = filtered_play.loc[filtered_play['Role'] == 'PR'].sort_values('Time')
                returner_df.rename(columns={'x':'ret_x', 
                                          'y': 'ret_y',
                                          'GSISID': 'ret_GSISID',
                                          'dis': 'ret_dis',
                                          'o': 'ret_o',
                                          'dir': 'ret_dir',
                                          'Role': 'ret_Role'
                                         }, inplace=True)
                returner_df = returner_df.drop('Event', axis = 1)
                
                cov_ret_df = coverage_df.merge(returner_df, how ='inner', on = ['Season_Year','GameKey','PlayID','Time'])
                cov_ret_df['dis_from_ret'] = ((cov_ret_df['cov_x'] -  cov_ret_df['ret_x']) ** 2 \
                                           + (cov_ret_df['cov_y'] -  cov_ret_df['ret_y']) ** 2).apply(np.sqrt)
                cov_ret_df['time_since_punt'] = cov_ret_df['Time'] - punt_event_time
                times_to_capture = [punt_event_time + pd.Timedelta(seconds=i) for i in range(1, 7)]
                cov_ret_df = cov_ret_df.loc[cov_ret_df['Time'].isin(times_to_capture)]
                cov_ret_df['gunner_blockers'] = gunner_blocker_count
                cov_ret_df['outcome'] = outcome
                cov_ret_df['injury'] = injury
                cov_ret_df['cov_speed'] = convert_to_mph(cov_ret_df.cov_dis, 20.455)
                cov_ret_df['ret_speed'] = convert_to_mph(cov_ret_df.ret_dis, 20.455)
                if len(cov_ret_df) > 0:
                    cov_ret_lst.append(cov_ret_df)
                    
    cov_ret_df = pd.concat(cov_ret_lst).reset_index(drop=True)          
    return cov_ret_df

In [86]:
def convert_to_mph(dis_vector, converter):
    mph_vector = dis_vector * converter
    return mph_vector

In [87]:
cov_ret_df = coverage_returner_space(pi, NGS_df)

In [88]:
cov_ret_df.gunner_blockers.value_counts(normalize=True)

2    0.452008
3    0.315041
4    0.214374
1    0.010546
0    0.008032
Name: gunner_blockers, dtype: float64

In [89]:
#Speed vs Number of Gunner Blockers at each second.

pd.pivot_table(cov_ret_df.loc[(cov_ret_df.cov_Role == 'GR') | (cov_ret_df.cov_Role == 'GL')], 
               index=['gunner_blockers'], values='cov_speed',
               columns=['time_since_punt'], aggfunc=[np.mean, np.median])

mean                                              \
time_since_punt   00:00:01   00:00:02   00:00:03   00:00:04   00:00:05   
gunner_blockers                                                          
0                12.207252  14.376943  14.873707  13.799258  10.994562   
1                16.478167  17.106088  16.039713  14.103184   8.113817   
2                17.688000  18.704544  17.637601  15.064499  12.696641   
3                16.188485  17.450438  17.401698  16.163206  14.145769   
4                14.598701  16.079530  16.354877  16.185452  14.298784   

                              median                                  \
time_since_punt   00:00:06  00:00:01   00:00:02   00:00:03  00:00:04   
gunner_blockers                                                        
0                      NaN  12.06845  15.034425  16.364000  14.31850   
1                      NaN  17.79585  18.818600  18.000400  15.95490   
2                13.155122  18.20495  19.432250  18.614051  16.15945   
3                10.747398  16.97765  18.000400  18.204950  16.97765   
4                12.324138  15.54580  16.773100  17.386750  16.56855   

                                       
time_since_punt   00:00:05   00:00:06  
gunner_blockers                        
0                10.636600        NaN  
1                 5.113750        NaN  
2                13.500301  13.500300  
3                15.136700   9.920675  
4                15.341250  13.091200

In [90]:
# Average distance for all player at the X second mark.  Return vs Fair Catch
pd.pivot_table(cov_ret_df, values='dis_from_ret', index=['time_since_punt'], columns=['outcome'], aggfunc=[np.mean, np.median])

mean                median           
outcome         fair_catch     return fair_catch     return
time_since_punt                                            
00:00:01         44.981770  48.471472  45.835058  48.650525
00:00:02         39.764362  43.661609  40.626422  43.601212
00:00:03         32.648800  37.529814  32.992786  37.158332
00:00:04         25.657960  31.941834  25.043261  31.169567
00:00:05         21.558572  27.815634  20.157205  26.088771
00:00:06         20.045922  22.920683  17.030813  20.193484

In [91]:
pd.pivot_table(cov_ret_df, values='dis_from_ret', index=['time_since_punt'], columns=['outcome','gunner_blockers'], aggfunc=np.mean)

outcome         fair_catch                                              \
gunner_blockers          0          1          2          3          4   
time_since_punt                                                          
00:00:01         47.026353  45.737942  44.068027  46.930285  46.565428   
00:00:02         43.257939  40.529952  38.909135  41.631461  41.003706   
00:00:03         37.062254  32.949803  32.065706  34.121274  32.869441   
00:00:04         29.816221  25.670866  25.555991  26.294769  24.706815   
00:00:05         23.292701  21.354172  22.111357  21.385757  20.327672   
00:00:06               NaN        NaN        NaN        NaN  20.045922   

outcome             return                                              
gunner_blockers          0          1          2          3          4  
time_since_punt                                                         
00:00:01         46.983556  48.445778  48.368493  48.675125  48.373827  
00:00:02         43.333499  44.139564  43.593242  43.744613  43.630136  
00:00:03         38.404080  37.964010  37.697907  37.514769  37.311463  
00:00:04         32.182807  32.015652  32.523455  31.754301  31.540063  
00:00:05         21.483868  31.668453  28.495432  27.391732  28.036701  
00:00:06               NaN        NaN  24.452747  20.625897  25.432733

In [92]:
#### At each second of each play... Who is the closest to the returner?  How far away are they?  Return vs Fair Catch

In [93]:
min_dist_from_ret = cov_ret_df.groupby(['Season_Year', 'GameKey','PlayID','time_since_punt'], as_index = False)['dis_from_ret'].min()

In [94]:
min_dist_from_ret.head()

,Season_Year,GameKey,PlayID,time_since_punt,dis_from_ret
0,2016,3,455,00:00:01,39.068560
1,2016,3,455,00:00:02,31.370344
2,2016,3,455,00:00:03,21.645880
3,2016,3,455,00:00:04,11.321985
4,2016,3,1542,00:00:01,38.607731


In [95]:
closest_df = cov_ret_df.merge(min_dist_from_ret.drop('time_since_punt', axis = 1), how = 'inner', on=['Season_Year', 'GameKey', 'PlayID', 'dis_from_ret'])

In [139]:
ex_cr = cov_ret_df.loc[(cov_ret_df.Season_Year==2016) & (cov_ret_df.GameKey== 289) \
                      & (cov_ret_df.PlayID== 2341) & (cov_ret_df.cov_GSISID== 32007)].sort_values('Time')

In [146]:
ex_cr.head()

,Season_Year,GameKey,PlayID,cov_GSISID,Time,cov_x,cov_y,cov_dis,cov_o,cov_dir,...,ret_o,ret_dir,ret_Role,dis_from_ret,time_since_punt,gunner_blockers,outcome,injury,cov_speed,ret_speed
83034,2016,289,2341,32007,2016-12-20 03:19:49.200,48.459999,39.070000,0.81,2.920000,101.209999,...,173.960007,205.169998,PR,38.630332,00:00:01,3,return,1,16.56855,2.65915
83042,2016,289,2341,32007,2016-12-20 03:19:50.200,56.730000,36.599998,0.95,40.200001,108.470001,...,176.490005,124.139999,PR,30.616338,00:00:02,3,return,1,19.43225,3.47735
83061,2016,289,2341,32007,2016-12-20 03:19:51.200,65.500000,33.520000,0.90,43.470001,106.339996,...,184.729996,67.169998,PR,22.786068,00:00:03,3,return,1,18.40950,2.45460
83065,2016,289,2341,32007,2016-12-20 03:19:52.200,73.760002,31.490000,0.74,48.580002,96.489998,...,188.220001,31.000000,PR,15.090299,00:00:04,3,return,1,15.13670,3.27280


In [145]:
#video_review.head(20)

In [142]:
x1 = np.array(ex_cr.cov_x)
y1 = np.array(ex_cr.cov_y)

x2 = np.array(ex_cr.ret_x)
y2 = np.array(ex_cr.ret_y)

In [143]:
y2

array([ 24.36000061,  22.84000015,  23.19000053,  23.93000031])

In [144]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

data=[dict(x=x1, y=y1, 
           mode='lines', 
           line=dict(width=2, color='blue')
          ),
      dict(x=x2, y=y2, 
           mode='lines', 
           line=dict(width=2, color='orange')
          )
    ]

layout = load_field()
layout['hovermode'] = 'closest'
layout['updatemenus'] = [{'type': 'buttons',
                           'buttons': [{'label': 'Play',
                                        'method': 'animate',
                                        'args': [None]}]}]

N = len(x1)
frames=[dict(data=[dict(x=[x1[k]], 
                        y=[y1[k]], 
                        mode='markers', 
                        marker=dict(color='red', size=10)
                        )
                  ]) for k in range(N)]
          
figure1=dict(data=data, layout=layout, frames=frames)
py.iplot(figure1)

In [96]:
#closest_df.groupby(['time_since_punt','cov_Role'], as_index=False)['dis_from_ret'].agg({'# of times closest': 'count','avg distance':np.mean}).sort_values('# of times closest', ascending =False)

In [97]:
pos_grouped_df = cov_ret_df.groupby(['cov_Role','time_since_punt', 'outcome'], as_index=False)['dis_from_ret'].mean()

In [46]:
trace1 = go.Bar(
        x=pos_grouped_df.loc[pos_grouped_df.outcome == 'fair_catch']['cov_Role'],
        y=pos_grouped_df.loc[pos_grouped_df.outcome == 'fair_catch']['dis_from_ret'],
        name='fair catch'
    )

trace2 = go.Bar(
        x=pos_grouped_df.loc[pos_grouped_df.outcome == 'return']['cov_Role'],
        y=pos_grouped_df.loc[pos_grouped_df.outcome == 'return']['dis_from_ret'],
        name='return'
    )

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    xaxis=dict(title='Position'),
    yaxis= dict(title='Avg Distance From Returner (Yds)')
)

fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

#### Punts Landing inside the 20

In [84]:
def parse_play_description(df, outcome_lst):
    parsed_df = play_info.loc[play_info.outcome.isin(outcome_lst),\
                             ['Season_Year','GameKey','PlayID',\
                              'PlayDescription','outcome']].reset_index(drop=True)
    punt_to_lst = []
    punt_dist_lst = []
    return_dist_lst = []
    punt_regex = '(punts [0-9]+ yards? to [A-Z]* [-+]?[0-9]+)| (punts [0-9]+ yards? to [-+]?[0-9]+)'
    return_regex = '(to [A-Z]* [0-9]+ for [-+]?[0-9]+ yards?)|(to [0-9]+ for [-+]?[0-9]+ yards?)|(ob at [A-Z]* [-+]?[0-9]+ for [-+]?[0-9]+ yards?)|(ob at [0-9]+ for [-+]?[0-9]+ yards?)|(for [-+]?[0-9]+ yards?, TOUCHDOWN)'
    
    for i in range(0, len(parsed_df)):
        punt_search = re.search(punt_regex, parsed_df.PlayDescription[i])
        return_search = re.search(return_regex, parsed_df.PlayDescription[i])
    
        if punt_search:
            punt_snip = re.findall(r'-?\d+', punt_search.group(0))
            if parsed_df.outcome[i] in ['downed','fair_catch', 'oob', 'return']:
                punt_to_lst.append(int(punt_snip[-1]))
                punt_dist_lst.append(int(punt_snip[0]))
            else:
                if parsed_df.outcome[i] == 'touchback':
                    punt_to_lst.append(0)
                    punt_dist_lst.append(int(punt_snip[0]))
                else:
                    print('Missing Punt Outcome at Row {}'.format(i))
        else:
            if parsed_df.outcome[i] == 'touchback':
                punt_to_lst.append(0)
                punt_dist_lst.append(int(punt_snip[0]))
            else:
                print('Missing Punt Outcome at Row {}'.format(i))
        
        if return_search:
            return_snip = re.findall(r'-?\d+', return_search.group(0))
            return_dist_lst.append(int(return_snip[-1]))
        else:
            if parsed_df.outcome[i] == 'touchback':
                return_dist_lst.append(20)
            elif parsed_df.outcome[i] in ['downed','fair_catch', 'oob']:
                return_dist_lst.append(0)
            elif 'no gain' in parsed_df.PlayDescription[i]:
                return_dist_lst.append(0)
            else:
                print('Missing Return Outcome at Row {}'.format(i))   
                
    parsed_df['punt_to'] = punt_to_lst
    parsed_df['punt_dist'] = punt_dist_lst
    parsed_df['return_dist'] = return_dist_lst
                                   
    return parsed_df

In [85]:
punt_detail_df = parse_play_description(play_info, ['touchback', 'fair_catch','oob', 'downed', 'return'])

In [80]:
#Fearing the extra 5 yard penalty, punters will be even more wary of avoiding a touchback.
#They will punt the ball shorter and with more hang time, or angle the ball out of bounds.

#From a returners perspective, if the ball is heading close to the goaline, they will be more likely to let it bounce 
#and take the touchback than return it because of the extra 5 yard bonus.

In [86]:
punt_detail_df.head()

,Season_Year,GameKey,PlayID,PlayDescription,outcome,punt_to,punt_dist,return_dist
0,2016,2,191,"(12:30) J.Hekker punts 52 yards to DAL 1, Center-J.McQuaide, downed by LA-E.Sims. Dallas challenged the kick downed ruling, and the play was REVERSED. J.Hekker punts 53 yards to end zone, Center-J.McQuaide, Touchback.",touchback,0,52,20
1,2016,2,1132,"(12:08) J.Hekker punts 51 yards to DAL 20, Center-J.McQuaide, fair catch by L.Whitehead. PENALTY on DAL-K.Wilber, Offensive Holding, 10 yards, enforced at DAL 20.",fair_catch,20,51,0
2,2016,2,1227,"(10:01) C.Jones punts 40 yards to LA 42, Center-L.Ladouceur. P.Cooper to DAL 33 for 25 yards (C.Jones; A.Hitchens).",return,42,40,25
3,2016,2,1864,"(:21) J.Hekker punts 31 yards to DAL 15, Center-J.McQuaide, out of bounds.",oob,15,31,0
4,2016,2,2247,"(10:26) M.Wile punts 40 yards to LA 45, Center-Z.Wood, out of bounds.",oob,45,40,0


In [67]:
punt_to_lst = []
punt_dist_lst = []
return_dist_lst = []

In [50]:
string = 'T.Morstead punts 47 yards to 17, Center-C.Highland. T.Ervin to HST 16 for -1 yards '

In [51]:
snip = re.search('(to [A-Z]* [0-9]+ for [-+]?[0-9]+ yards?)', string).group(0)